<a href="https://colab.research.google.com/github/talhaanwarch/Plant-Leaf-Disease-Classification/blob/master/stratified_aug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone https://github.com/mjkvaak/ImageDataAugmentor.git

Cloning into 'ImageDataAugmentor'...
remote: Enumerating objects: 98, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 263 (delta 54), reused 38 (delta 15), pack-reused 165
Receiving objects: 100% (263/263), 130.33 KiB | 285.00 KiB/s, done.
Resolving deltas: 100% (153/153), done.


In [0]:
%%capture
!wget https://data.mendeley.com/datasets/3f83gxmv57/2/files/53398b67-6f0e-4a67-8384-e2b574b2ebf4/Citrus.zip

In [0]:
%%capture
!unzip /content/Citrus.zip

In [0]:
%%capture
!pip install efficientnet

In [0]:
import glob
import pandas as pd
import os
import numpy as np

In [0]:

Black_spot=os.listdir('/content/Citrus/Leaves/Black spot')
Melanose=os.listdir('/content/Citrus/Leaves/Melanose')
canker=os.listdir('//content/Citrus/Leaves/canker')
greening=os.listdir('/content/Citrus/Leaves/greening')
healthy=os.listdir('/content/Citrus/Leaves/healthy')


In [0]:
def encode(category,category_name):
  category=list(zip(category,['{}'.format(category_name)]*(len(category))))
  category_df = pd.DataFrame(category,columns=['file','label']) 
  return category_df



In [0]:
Black_spot_df=encode(Black_spot,'Black_spot')
Melanose_df=encode(Melanose,'Melanose')
canker_df=encode(canker,'canker')
greening_df=encode(greening,'greening')
healthy_df=encode(healthy,'healthy')

In [0]:
dataframe=pd.concat([Black_spot_df,Melanose_df,canker_df,greening_df,healthy_df])
dataframe.head()

,file,label
0,b (98).png,Black_spot
1,b (273).png,Black_spot
2,b (56).png,Black_spot
3,b (84).png,Black_spot
4,b (222).png,Black_spot


In [0]:
dataframe['label'].value_counts()

greening      204
Black_spot    171
canker        163
healthy        58
Melanose       13
Name: label, dtype: int64

In [0]:
dataframe=dataframe.sample(frac=1)

In [0]:
!mkdir data

In [0]:
!cp -r /content/Citrus/Leaves/Black\ spot/* data
!cp -r /content/Citrus/Leaves/Melanose/* data
!cp -r /content/Citrus/Leaves/canker/* data
!cp -r /content/Citrus/Leaves/healthy/* data
!cp -r /content/Citrus/Leaves/greening/* data


cp: cannot stat '/content/Citrus/Leaves/greeming/*': No such file or directory


In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from albumentations import *


In [0]:
aug=Compose([
        RandomRotate90(),
        Flip(),
        Transpose(),
        OneOf([
            IAAAdditiveGaussianNoise(),
            GaussNoise(),
            
        ], p=0.2),
        OneOf([
            MotionBlur(p=.2),
            MedianBlur(blur_limit=3, p=.1),
            Blur(blur_limit=3, p=.1),
        ], p=0.3),
        ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=.2),
        OneOf([
            OpticalDistortion(p=0.3),
            GridDistortion(p=.1),
            IAAPiecewiseAffine(p=0.3),
        ], p=0.3),
        OneOf([
            CLAHE(clip_limit=2),
            IAASharpen(),
            IAAEmboss(),
            RandomContrast(),
            RandomBrightness(),
        ], p=0.3),
        #HueSaturationValue(p=0.3),
    ], p=1)

In [0]:
#from keras_preprocessing.image import ImageDataGenerator
from ImageDataAugmentor.image_data_augmentor import *
data_gen= ImageDataAugmentor(
    #horizontal_flip=True,
    #vertical_flip=True,
    #rotation_range=90,
    # width_shift_range=0.2,
    # height_shift_range=0.2,
    # zoom_range=.2,
    rescale=1/255,
    # fill_mode='nearest',
    # shear_range=0.2,
    # brightness_range=[0.2, 2.0],
    augment =aug)

Using TensorFlow backend.


In [0]:
img_shape=380
batch_size=16

In [0]:
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPool2D, BatchNormalization,Input,MaxPooling2D,GlobalMaxPooling2D,concatenate
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K
import tensorflow as tf

In [0]:
df_images=dataframe['file']
df_labels=dataframe['label']
col=['Black_spot','Melanose','canker','greening','healthy']

In [0]:
from sklearn.model_selection import KFold,StratifiedKFold
import gc

from sklearn.metrics import classification_report,accuracy_score,f1_score
classification_reports=[]
kf = StratifiedKFold(n_splits=5)
history=[]
accuracy=[]
f1_macro_scores=[]
f1_weighted_scores=[]
fold=0
for train_index, test_index in kf.split(df_images,df_labels):
  train=df.iloc[train_index,:]
  val=df.iloc[test_index,:]

  print('-----------fold {}--------------'.format(fold))
  train_generator=data_gen.flow_from_dataframe(train,directory='data',
                                                      target_size=(img_shape,img_shape),
                                                      x_col="file",
                                                      y_col=col,
                                                      class_mode='raw',
                                                      shuffle=True,
                                                      batch_size=batch_size)

  val_generator=data_gen.flow_from_dataframe(val,directory='data',
                                                        target_size=(img_shape,img_shape),
                                                        x_col="file",
                                                        y_col=col,
                                                        class_mode='raw',
                                                        shuffle=False,
                                                        batch_size=batch_size)
  #train_generator.next()[0].shape,train_generator.next()[1].shape
  import efficientnet.tfkeras as efn
  model =efn.EfficientNetB4(weights ='noisy-student', include_top=False, input_shape = (img_shape,img_shape,3))
    
  x = model.output
  x = GlobalAveragePooling2D()(x)
  x = Dropout(0.3)(x)
  x = Dense(128, activation="relu")(x)
  #x = Dropout(0.3)(x)
  x = Dense(64, activation="relu")(x)
  predictions = Dense(5, activation="softmax")(x)
  model = Model(inputs=model.input, outputs=predictions)

  def custom_loss(y_true, y_pred):
      return tf.keras.losses.categorical_crossentropy(y_true, y_pred, label_smoothing=0.1)
  model.compile(optimizer=Adam(0.0001), loss=custom_loss, metrics=['accuracy'])

  results = model.fit(train_generator,epochs=25,
                              steps_per_epoch=train_generator.n/batch_size,
                              validation_data=val_generator,
                             validation_steps=val_generator.n/batch_size,
                              callbacks=[ReduceLROnPlateau(monitor='val_loss', factor=0.5,patience=3, min_lr=0.000001),
                                         ModelCheckpoint('model_{}.hdf5'.format(fold), save_best_only=True, monitor='val_loss', mode='min')
                                         ])
  model.load_weights('model_{}.hdf5'.format(fold))
  val_generator.reset()
  y_pred=model.predict(val_generator,steps=val_generator.n/batch_size,verbose=1)
  y_pred=y_pred.round().astype(int)
  y_true=val.iloc[:,2::]
  classification_reports.append(classification_report(y_true, y_pred,target_names=col))#
  accuracy.append(accuracy_score(y_true,y_pred))
  f1_macro=f1_score(y_true,y_pred,average='macro')
  print('f1_macro score is ', f1_macro)
  f1_macro_scores.append(f1_macro)

  f1_weighted=f1_score(y_true,y_pred,average='weighted')
  print('f1_weighted score is ', f1_weighted)
  f1_macro_scores.append(f1_weighted)

  fold+=1
  del model
  tf.keras.backend.clear_session()
  gc.collect()
  

-----------fold 0--------------
Found 487 validated image filenames.
Found 122 validated image filenames.
Epoch 1/25
31/30 [==============================] - 42s 1s/step - loss: 1.4310 - accuracy: 0.4251 - val_loss: 1.5049 - val_accuracy: 0.2951 - lr: 1.0000e-04
Epoch 2/25
31/30 [==============================] - 41s 1s/step - loss: 1.1575 - accuracy: 0.6653 - val_loss: 1.3545 - val_accuracy: 0.4918 - lr: 1.0000e-04
Epoch 3/25
31/30 [==============================] - 41s 1s/step - loss: 0.9083 - accuracy: 0.7556 - val_loss: 1.2866 - val_accuracy: 0.5656 - lr: 1.0000e-04
Epoch 4/25
31/30 [==============================] - 40s 1s/step - loss: 0.8052 - accuracy: 0.7844 - val_loss: 1.2189 - val_accuracy: 0.6475 - lr: 1.0000e-04
Epoch 5/25
31/30 [==============================] - 38s 1s/step - loss: 0.7403 - accuracy: 0.8172 - val_loss: 1.0673 - val_accuracy: 0.8033 - lr: 1.0000e-04
Epoch 6/25
31/30 [==============================] - 39s 1s/step - loss: 0.6912 - accuracy: 0.8090 - val_loss:

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


f1_macro score is  0.7936437523295037
f1_weighted score is  0.8309799352201519
-----------fold 1--------------
Found 487 validated image filenames.
Found 122 validated image filenames.
Epoch 1/25


ResourceExhaustedError: ignored

In [0]:
val.iloc[0,:]

Black_spot    0
Melanose      0
canker        0
greening      1
healthy       0
Name: 70, dtype: uint8

In [0]:
accuracy

[0.9827882960413081,
 0.9845094664371773,
 0.9724612736660929,
 0.9724612736660929,
 0.9741824440619621]

In [0]:
print(np.mean(accuracy))

0.9772805507745266


In [0]:
f1_scores

[0.9815409599130224,
 0.9861540631747995,
 0.9772288842920812,
 0.9806688882043814,
 0.978913715784171]

In [0]:
np.mean(f1_scores)

0.980901302273691

In [0]:

for i in range(5):
  print('--------------FOLD {}------------------'.format(i))
  print(classification_reports[i])

--------------FOLD 0------------------
              precision    recall  f1-score   support

       covid       1.00      0.96      0.98        47
      normal       0.97      0.99      0.98       265
   pneumonia       0.99      0.98      0.99       269

   micro avg       0.98      0.98      0.98       581
   macro avg       0.99      0.98      0.98       581
weighted avg       0.98      0.98      0.98       581
 samples avg       0.98      0.98      0.98       581

--------------FOLD 1------------------
              precision    recall  f1-score   support

       covid       1.00      0.98      0.99        43
      normal       0.98      0.99      0.98       259
   pneumonia       0.99      0.98      0.99       279

   micro avg       0.99      0.98      0.99       581
   macro avg       0.99      0.98      0.99       581
weighted avg       0.99      0.98      0.99       581
 samples avg       0.98      0.98      0.98       581

--------------FOLD 2------------------
             